In [1]:
# Check core SDK version number
import azureml.core

print( "SDK version:" , azureml.core.VERSION )

SDK version: 1.0.21


In [ ]:
from azureml.core import Workspace

# Create the workspace using the specified parameters
ws = Workspace.from_config()
print( ws.name , ws.resource_group , ws.location , ws.subscription_id , sep = '\n')

In [3]:
import os
from azureml.core import Experiment

# choose a name for the run history container in the workspace
experiment_name = "hack"

exp = Experiment( ws , name=experiment_name )

# project folder name - the folder that contains all training scripts
project_folder = os.path.join( os.getcwd() , experiment_name )
print( project_folder )

C:\Users\delbertm\notebooks\hack


In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

cluster_name = 'hackcluster'

try:
    compute_target = ComputeTarget( workspace=ws, name=cluster_name )
    print( 'Found existing compute target' )
except ComputeTargetException:
    print( 'Creating a new compute target...' )
    compute_config = AmlCompute.provisioning_configuration( vm_size = "STANDARD_D2_V2" ,
                                                                min_nodes = 0 ,
                                                                max_nodes = 7 )

    # create the cluster
    compute_target = ComputeTarget.create( ws , cluster_name, compute_config )

    compute_target.wait_for_completion( show_output=True )

print( compute_target.status.serialize() )

Found existing compute target
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-04-01T22:27:52.572000+00:00', 'errors': None, 'creationTime': '2019-04-01T21:59:33.795858+00:00', 'modifiedTime': '2019-04-01T22:00:09.485643+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 7, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [22]:
from azureml.train.estimator import Estimator

est = Estimator ( source_directory = project_folder ,
                  compute_target = compute_target ,
                  node_count = 3 ,
                  distributed_backend = "mpi" ,
                  entry_script = 'launch.py' ,
                  pip_packages = [ "ray==0.6.5" , "psutil" , "setproctitle" ] )

In [31]:
run = exp.submit( est )
print( run )

Run(Experiment: hack,
Id: hack_1554239111_1206a781,
Type: azureml.scriptrun,
Status: Queued)


In [32]:
from azureml.widgets import RunDetails

RunDetails( run ).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
#cancel the current run
run.cancel()